# This is just a notebook to visualise 1kHz filtered raw data

## Setup everything

### Import packages

In [ ]:
import os
import sys
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
from open_ephys.analysis import Session

import mmap
import xarray as xr
import dask.array as da
#%matplotlib widget

from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter


from ipyfilechooser import FileChooser
import ipywidgets as widgets
import pickle
import configparser
import ast

### Import local config, create it if inexistant
Local congig file will be a file named localConfig.ini and stored in the python subfolder of AudreyHayLab repo that will be ignored during commits and will store all user-specific variables (projects, defautl path to files...). If the file does not exist at beginning, it is created with default values that can be modified.

In [ ]:
def generateLocalConfigFile(configFN):
    config = configparser.ConfigParser()
    config['DATA'] = {'path': os.path.expanduser("~")}
    config['ANALYSIS'] = {
        'path': os.path.join(os.path.expanduser("~"),'Analysis'),
        'projecttype': 0,
        'animalid': 0,
        'projectid': 'AProject',
        'subprojectid': 'OneOfItsSubProject',
        'conditionid': 'control',
        'recordingID': 0,
        'suffix': ''
        }
    config['AProject.OneOfItsSubProject'] = {
        'design': 0,
        'nAnimal': 6,
        'conditions': ["control"],
        'nrecordings': 1
        }
    with open(configFN, 'w') as configfile:
        config.write(configfile)
    return config

In [ ]:
configFN = 'localConfig.ini'

if os.path.isfile(configFN):
    config = configparser.ConfigParser()
    config.read(configFN)
else:
    config = generateLocalConfigFile(configFN)

rawDataPath = config['DATA']['path']

def updateConf():
    with open(configFN, 'w') as configfile:
        config.write(configfile)

def updateDict(dictName, key, value):
    with open(dictName, 'rb') as f:
        loaded_dict = pickle.load(f)
    with open(dictName, 'wb') as f:
        loaded_dict[key] = value
        pickle.dump(loaded_dict,f)

## This is not implemented yet
This section defines functions that will be used to load the parameters for each experiment so that the script is truely independant of experiment while can be reproduced reliably

### Define usefull functions to deal with config dictionaries
channelsMap is a dictionary with one key per brain region and/or canal of interest (TTL). The value associated to the key is an array with every corresponding canal. The "status" key defines which of these canals to use for analyses:
- 0: not to use
- 1 only: floating point canal
- 1 and 2: differential signal 2-1

In [ ]:
def getPathComponent(filename,projectType):
    
    dirPathComponents = os.path.normpath(filename).split(os.sep)
    expeInfo = dict()

    expeInfo['analysisPath'] = os.path.join('/',*dirPathComponents[0:-5])
    expeInfo['ProjectID'] = dirPathComponents[-5]
    expeInfo['subProjectID'] = dirPathComponents[-4]

    projectConfig = os.path.join('/',*dirPathComponents[0:-3],'projectConfig.pkl')
    if os.path.isfile(projectConfig):
        with open(projectConfig, 'rb') as f:
            loaded_dict = pickle.load(f)
            expeInfo['projectType'] = loaded_dict['projectType']
    else:
        with open(projectConfig, 'wb') as f:
            projDict = dict(projectType = projectType)
            pickle.dump(projDict, f)
            print('Project config dict created')

    if projectType == 0:
        expeInfo['conditionID'] = dirPathComponents[-3]
        expeInfo['AnimalID'] = dirPathComponents[-2]
    else:
        expeInfo['AnimalID'] = dirPathComponents[-3]
        expeInfo['conditionID'] = dirPathComponents[-2]
        
    expeInfo['recordingID'] = dirPathComponents[-1]

    return expeInfo


In [ ]:
expeInfo = dict()

projects = [p.split('.')[0] for p in config.sections() if p not in ['DATA','ANALYSIS']]

subprojects = {p.split('.')[0]: widgets.Dropdown(
    options=[p.split('.')[1]],
    description='Sub-project (you can update the list in your localConfig.ini file):',
    ) for p in config.sections() if p not in ['DATA','ANALYSIS']} 


def printExpeInfo(**func_kwargs):
    pass#print(expeInfo)

def updateProject(widget):
    ProjectID = widget.new
    expeInfo['ProjectID'] = ProjectID
    new_i = widgets.interactive(printExpeInfo, project=wProject, subProject=subprojects[ProjectID])
    i.children = new_i.children


def updateSubProject(widget):
    if widget['type'] == 'change' and widget['name'] == 'value':
        expeInfo['subProjectID'] = widget.new

def update_design(widget):
    projectType = widget.new
    if projectType == 0:
        new_i = widgets.interactive(printExpeInfo, project=wProject, subProject=subprojects[ProjectID], design=wDesign, condition=wCondition, animal=wAnimal, rec=wRec)
    else:
        new_i = widgets.interactive(printExpeInfo, project=wProject, subProject=subprojects[ProjectID], design=wDesign, animal=wAnimal, condition=wCondition, rec=wRec)
    i.children = new_i.children
    %store projectType

analysisPath = config['ANALYSIS']['path']
projectType = int(config['ANALYSIS']['projectType'])
ProjectID = config['ANALYSIS']['ProjectID']
subProjectID = config['ANALYSIS']['subProjectID']
conditionID = config['ANALYSIS']['conditionID']
AnimalID = int(config['ANALYSIS']['AnimalID'])
recordingID = int(config['ANALYSIS']['recordingID'])

wProject = widgets.Dropdown(
    options=projects,
    value=ProjectID,
    description='Project (you can update the list in your localConfig.ini file):',
    disabled=False,
)
wProject.observe(updateProject, 'value')

#wSubProject.observe(updateSubProject)

designs = ['independant groups', 'within subject']
wDesign = widgets.RadioButtons(
    options=designs,
    value=designs[projectType], # Defaults to 'independant groups'
    description='Experiment design:'
)
wDesign.observe(update_design, names=['index'])

wAnimal = widgets.BoundedIntText(
    value=AnimalID,
    min=0,
    #max=10,
    step=1,
    description='Animal ID:'
)

conditions = ast.literal_eval(config["{}.{}".format(ProjectID, subProjectID)]['conditions'])
wCondition = widgets.Dropdown(
    options=conditions,
    value=conditionID,
    description='Condition:',
)

wRec = widgets.BoundedIntText(
    value=recordingID,
    min=0,
    #max=10,
    step=1,
    description='Recording ID:'
)

if projectType == 0:
    i = widgets.interactive(printExpeInfo, project=wProject, subProject=subprojects[ProjectID], design=wDesign, condition=wCondition, animal=wAnimal, rec=wRec)
else:
    i = widgets.interactive(printExpeInfo, project=wProject, subProject=subprojects[ProjectID], design=wDesign, animal=wAnimal, condition=wCondition, rec=wRec)


def defineExpeInfo():
    display(i)

In [ ]:
def generateConfigDict(filename, rawDataPath = None):
    
    numChanels=64

    channelsMap = dict( \
        EMG = [dict(canal = 6, status=1)],
        PFC = [dict(canal = 5, status=1),
            dict(canal = 4, status=2)
            ],
        CA1 = [dict(canal = 8, status=1),
            dict(canal = 0, status=0),
            dict(canal = 1, status=0),
            ],
        TTL = [dict(canal = 10, status=1)],
    )

    projectType = int(config['ANALYSIS']['projectType'])
    expeInfo = getPathComponent(filename,projectType)

    allParamsDict = dict(channelsMap = channelsMap, numChanels = numChanels, rawDataPath = rawDataPath, expeInfo = expeInfo)

    with open(filename, 'wb') as f:
        pickle.dump(allParamsDict, f)
            

In [ ]:
def loadConfigDict(filename):
    global numChanels, rawDataPath, expeInfo, channelsMap
    with open(filename, 'rb') as f:
        loaded_dict = pickle.load(f, encoding='UTF8')
        numChanels = loaded_dict['numChanels']
        rawDataPath = loaded_dict['rawDataPath']
        if 'expeInfo' in loaded_dict:
            expeInfo = loaded_dict['expeInfo']
        else:
            expeInfo = getPathComponent(filename,projectType)
            updateDict(filename,'expeInfo',expeInfo)
        channelsMap = loaded_dict['channelsMap']

## Choose experiment
Select the experiment to display. If the experiment was already analyzed, a saved_dictionary.pkl was created and contains all necessary variables. Select this file. Otherwise select the raw data recording file.
>**If you have a file with channel mapping somewhere**, we should make sure it is properly translated into a dict.pkl

In [ ]:
currentFile = None
%store -r currentFile
print(currentFile)

if currentFile is not None and os.path.isfile(currentFile): # a file is currently being used
    pathName, fileName = os.path.split(currentFile)
    loadConfigDict(currentFile)
else:
    pathName = rawDataPath
    fileName = ""
    
fc = FileChooser(path=pathName, filename=fileName, select_default=True, show_only_dirs = False, title = "<b>Select file</b>")
display(fc)

def update_my_folder(chooser):
    global currentFile
    selection = chooser.selected
    if selection.endswith("pkl"):
        currentFile = str(selection)
        loadConfigDict(selection)
    else:
        print("this is not a config file and we should deal with that")
        defineExpeInfo()
        if projectType == 0:
            path = os.path.join(analysisPath, ProjectID, subProjectID, conditionID, str(AnimalID), str(recordingID))
            
        else:
            path = os.path.join(analysisPath, ProjectID, subProjectID, str(AnimalID), conditionID, str(recordingID))
        os.makedirs(path, exist_ok=True)
        currentFile = os.path.join(os.path.split(path)[0],'saved_dictionary.pkl')
        generateConfigDict(currentFile, rawDataPath = selection)
        loadConfigDict(currentFile)
    %store currentFile
    
# Register callback function
fc.register_callback(update_my_folder)

### Possibility to change raw data path 
if for some reason the path to the raw data is wrong, you can update it here

In [ ]:
#print(rawDataPath)
if rawDataPath is not None:
    rawDirname, rawFN = os.path.split(rawDataPath)
    rfc = FileChooser(path=rawDirname, filename=rawFN,select_default=True, show_only_dirs = False, title = "<b>ePhys data</b>")
else:
    rfc = FileChooser(show_only_dirs = False, title = "<b>ePhys data</b>")
display(rfc)

# Sample callback function
def update_rawDataPath(chooser):
    global rawDataPath
    rawDataPath = chooser.selected
    #updateDict(currentFile, 'rawDataPath', rawDataPath)

# Register callback function
rfc.register_callback(update_rawDataPath)

## Load Data

### Map the whole data into memory

In [ ]:
rawDataDir, rawFileBaseName = os.path.split(rawDataPath)

if rawFileBaseName == "continuous.dat":
    All = np.memmap(rawDataPath, mode='r', dtype='int16')
    #All = np.fromfile(filename, dtype="int16")
    All = All.reshape(-1,numChanels)
elif rawFileBaseName.endswith(".npy"):
    All = np.load(rawDataPath, mmap_mode= 'r')

In [ ]:
filenameT = os.path.join(rawDataDir,"timestamps.npy")
if os.path.isfile(filenameT):
    Timestamps = np.load(filenameT)
    Timestamps.shape
    Timestamps = Timestamps*2000
    Timestamps = Timestamps.astype(int)

### Extract submatrix of interest

In [ ]:
start = 000000
end = All.shape[0]
combined = np.empty((end-start,0),np.int16)
channelLabels = []
for region in channelsMap:
    print(region, "->", channelsMap[region])
    if len([canal["canal"] for canal in channelsMap[region] if canal["status"]==2])>0:
        c2 = [canal["canal"] for canal in channelsMap[region] if canal["status"]==2][0]
        c1 = [canal["canal"] for canal in channelsMap[region] if canal["status"]==1][0]
        print("Getting differential signal of channel {} - channel {} for {}".format(c2,c1,region))
        channelLabels.append(region)
        combined = np.append(combined, All[start:end, c2, np.newaxis] - All[:, c1, np.newaxis], axis=1)
    elif len([canal["canal"] for canal in channelsMap[region] if canal["status"]==1])>0:
        c = [canal["canal"] for canal in channelsMap[region] if canal["status"]==1][0]
        print("Getting floating signal of channel {} for {}".format(c,region))
        combined = np.append(combined, All[start:end,c, np.newaxis], axis=1)
        channelLabels.append(region)

In [ ]:
%gui qt # allows the app to be closed clean and reopen
app = mkQApp()


sample_rate = 1000.
t_start = 0.

TTL = Timestamps

#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: TTL, 1: TTL}
#and asign them to some channels each
scatter_channels = {0: [0, 12], 1: [0, 1]}
source = AnalogSignalSourceWithScatter(All, sample_rate, t_start, scatter_indexes, scatter_channels)


#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(combined, sample_rate, t_start, 'Signals', channel_names=channelLabels)
#view1 = TraceViewer(source=source)
win.add_view(view1)

#Parameters can be set in script
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

cmap = matplotlib.colormaps["hsv"]#Wistia"]
nCh = len(view1.by_channel_params.children())
for ch in range(nCh):
    #view1.by_channel_params[f'ch{ch}', 'gain'] = 0.00002
    #view1.by_channel_params[f'ch{ch}', 'offset'] = 0.1
    view1.by_channel_params[f'ch{ch}', 'color'] = matplotlib.colors.to_hex(cmap(ch/nCh), keep_alpha=False)
    pass


#Run
win.show()
#app.exec()  #if commented, the app is shown and fonctionnal. Maybe detecting buttons. the Python icon doesn't close any better

: 